In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

In [2]:

#Get a range of the column values so they can be used as the column names
def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

def initialLoad(directory):
    #Read the csv file, discard the null columns
    turbo = pd.read_csv(directory, header=None, sep=' ')
    turbo = turbo.drop([26, 27], axis=1)
    #Drop the simulation and cycle count from the data
    turboSens = turbo.drop([0,1,2,3,4], axis=1)
    turboSens.columns = getColumns(turboSens.shape[1])
    return turbo, turboSens


#Read the csv file, discard the null columns
directory = "../CMAPSSData/train_FD001.txt"
directoryTest = "../CMAPSSData/test_FD001.txt"
directoryTestExtra = "../CMAPSSData/RUL_FD001.txt"

turbo,turboSens = initialLoad(directory)

valTurbo,valSens = initialLoad(directoryTest)


In [3]:
minVals = []
maxVals = []
# Get the minimum and maximum value of each column and store them in minVals and maxVals respectively
for i in range(len(turboSens.columns)):
    minVals.append(turboSens[i].min())
    maxVals.append(turboSens[i].max())
    

In [4]:
#Get  [max value - min value]
maxNormed = []
for mini,maxi in zip(minVals,maxVals):
    val = maxi - mini
    maxNormed.append(val)
print(maxNormed)

#Get a list of the zero values in max normed(since these columns are probably not useful)
zeroes = []
for i in range(len(maxNormed)):
    if maxNormed[i] == 0:
        zeroes.append(i)

[0.0, 3.3199999999999363, 45.87000000000012, 59.24000000000001, 0.0, 0.00999999999999801, 6.209999999999923, 0.6599999999998545, 222.86000000000058, 0.0, 1.6799999999999997, 4.689999999999941, 0.6799999999998363, 193.77999999999975, 0.2599, 0.0, 12, 0, 0.0, 1.2899999999999991, 0.7241999999999997]


In [5]:
print(zeroes)

[0, 4, 9, 15, 17, 18]


In [6]:
#Drop the data columns that match the zero values in maxNormed
turboUsefulSens = turboSens
#turboUsefulSens.columns = getColumns(turboUsefulSens.shape[1])

valUsefulSens = valSens
#valUsefulSens.columns = getColumns(valUsefulSens.shape[1])



In [7]:
turboNp = turboUsefulSens.to_numpy()
valNp = valUsefulSens.to_numpy()


In [8]:
#Get the number of cycles for each simulation, and split each simulation
maxCycles = []
simCount = 100

def getTargets(cycleList, count):
    simulations = []
    current = 0
    for i in range(simCount):
        sim = [cycleList[current]]
        current+=1
        while current<len(cycleList):
            if(cycleList[current]==1):
                break
            sim.append(cycleList[current])
            current+=1
        simulations.append(sim)
    return simulations

simulations = getTargets(turbo[1], simCount)


In [9]:
#Get the lowest cycle max
maxCycles = []
for i in simulations:
    maxCycles.append(max(i))
    
print(min(maxCycles))

128


In [10]:
#Flip the cycle counts so they represent deterioration instead of incrementing(decrementing from total cycles to 0 instead of incrementing from 0 to total cycles)
simulationsRev = []
for sim in simulations:
    simRev = []
    maxSim = max(sim)
    for i in sim:
        simRev.append(maxSim - i)
    simulationsRev.append(simRev)

In [11]:
flat_list = []
for sim in simulationsRev:
    for item in sim:
        flat_list.append(item)
        
flatNp = np.array(flat_list)

In [12]:
np.save("Train", turboNp)
np.save("TrainTarget", flatNp)

In [13]:


valTargets = getTargets(valTurbo[1],simCount)
file = open(directoryTestExtra, 'r')

valTargetRev = []
for sim in valTargets:
    rev = []
    maxCount = max(sim)
    extra = int(file.readline())
    for i in sim:
        rev.append(maxCount - i + extra)
    valTargetRev.append(rev)
file.close()

flat_val = []
for sim in valTargetRev:
    for item in sim:
        flat_val.append(item)
        
        
valTargNp = np.array(flat_val)

In [14]:
np.save("Validate", valNp)
np.save("ValidateTarget", valTargNp)

In [15]:
print(len(valNp))
print(len(valTargNp))

print(len(turboNp))
print(len(flatNp))

13096
13096
20631
20631


In [16]:
allSensNp = np.append(turboNp, valNp, axis=0)
allTargNp = np.append(flatNp, valTargNp)

print(len(allSensNp))
print(len(allTargNp))

33727
33727


In [17]:
np.save("all", allSensNp)
np.save("allTarget", allTargNp)